## Installations & Downloads

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import pickle
import scipy
import pandas as pd
from scipy import stats
from scipy.stats import normaltest
import matplotlib as mpl
import itertools
from itertools import groupby

sys.path.append('/sise/home/shakarch/muscle-formation-regeneration')
sys.path.append(os.path.abspath('..'))

import warnings
warnings.simplefilter('ignore', pd.errors.DtypeWarning)
np.seterr(divide='ignore')
warnings.simplefilter("ignore")


import TimeSeriesAnalysis.analysis.calc_single_cell_properties as calcultator 
import TimeSeriesAnalysis.analysis.utils as utils
import TimeSeriesAnalysis.data_layer.utils as data_utils
import TimeSeriesAnalysis.configuration.consts as consts
import TimeSeriesAnalysis.configuration.params as params

# Load the data

In [3]:
scores_df_s3 = data_utils.get_scores_df_with_fusion()

In [ ]:
# validate that all fusion-timing-tagged-cells are semi-manually tracked
no_data_tracks = []
untracked_cells_ids = []
for track_id in scores_df_s3["Spot track ID"].unique():
    track_data = scores_df_s3[(scores_df_s3["Spot track ID"]==track_id)]
    if len(track_data)<=1:
        if track_data["manual manual"].sum()==0:
            untracked_cells_ids.append(track_id)
        else:
            no_data_tracks.append(track_id)
        
print("number of fusion annotated cells: ", scores_df_s3["Spot track ID"].nunique())
print(f"number of untracked cells: {len(untracked_cells_ids)}/{scores_df_s3['Spot track ID'].nunique()}")
print(f"number of no data cells: {len(no_data_tracks)}/{scores_df_s3['Spot track ID'].nunique()}")
print(f"no data cells: {no_data_tracks}")

# Inspect each annotated cell

In [ ]:
diff_score_1st_fusion = scores_df_s3[(scores_df_s3["time"]==scores_df_s3["fusion_time"])]
print(f"average differentiation score on fusion (motility): {round(diff_score_1st_fusion['score_motility'].mean(),3)} +- {round(diff_score_1st_fusion['score_motility'].std(),3)}")
print(f"average differentiation score on fusion (intensity): {round(diff_score_1st_fusion['score_intensity'].mean(),3)} +- {round(diff_score_1st_fusion['score_intensity'].std(),3)}")

In [ ]:
scores_df_s3_fused = calcultator.get_terminal_diff_time(scores_df_s3_fused, modal_col, 0.78)
scores_df_s3_fused = calcultator.get_differentiation_fusion_duration(scores_df_s3_fused, modal_col)

(low_thresh, high_thresh) = ((0, 0.5), (0.5, 1)) if "binary" in modal_col else ((0.2, 0.3), (0.7, 0.8))
scores_df_s3_fused = calcultator.get_diff_duration(scores_df_s3_fused, modal_col, low_thresh, high_thresh)
scores_df_s3_fused = calcultator.get_diff_start_time(scores_df_s3_fused, modal_col, low_thresh, high_thresh)